In [ ]:
import logging
import requests
import pyodbc
import os
import io
from datetime import datetime, date, timedelta
import time
import json
import gzip
import pandas as pd
import traceback
import numpy as np

idag = datetime.now()
igår = idag - timedelta(days=1)
år = idag.year
måned = idag.month
dag = idag.day
if måned <= 7:
    termin = "VÅR"
else:
    termin = "HØST"

CD2_base_url = "https://api-gateway.instructure.com"
CD2_client_id = os.environ['CD2_client_id']
CD2_client_secret = os.environ['CD2_client_secret']
conn_str = os.environ["Connection_SQL"]

: 

In [5]:
def akv_finn_sist_oppdatert(tabell):
    """
    Returner den siste oppdateringstida for den gitte tabellen fra akv_sist_oppdatert-tabellen.
    Hvis ingen dato er gitt (eller vi ikkje får kontakt med databasen), returner igår.
    """
    try:
        with pyodbc.connect(conn_str) as connection:
            cursor = connection.cursor()
            query = """
            SELECT [sist_oppdatert] FROM [dbo].[akv_sist_oppdatert]
            WHERE [tabell] = ?
            """
            cursor.execute(query, (tabell,))
            row = cursor.fetchone()
            if row:
                logging.debug(f"{tabell} er sist oppdatert (Azure): {row[0].isoformat() + 'Z'}")
                return row[0].isoformat() + "Z"
            
    except pyodbc.Error as exc:
        logging.debug(f"{tabell} er sist oppdatert (lokal): {(date.today() - timedelta(days=1)).isoformat() + 'Z'}") 
        return (date.today() - timedelta(days=1)).isoformat() + "Z"


def akv_lagre_sist_oppdatert(tabell, dato):
    """
    Lagre datoen for siste oppdatering av tabell i Azure eller lokalt (dersom vi ikkje får kontakt med databasen).
    """
    
    try:
        with pyodbc.connect(conn_str) as conn:
            cursor = conn.cursor()
            query = """
            MERGE INTO [dbo].[akv_sist_oppdatert] AS target 
            USING (VALUES (?, ?)) AS source (tabell, sist_oppdatert) 
            ON target.[tabell] = source.[tabell]
            WHEN MATCHED THEN
                UPDATE SET target.[sist_oppdatert] = source.[sist_oppdatert]
            WHEN NOT MATCHED THEN
                INSERT ([tabell], [sist_oppdatert]) VALUES (source.[tabell], source.[sist_oppdatert]);
            """ 
            cursor.execute(query, (tabell, dato))
            conn.commit()
            logging.debug(f"{tabell} er sist oppdatert (Azure): {dato}")
    except pyodbc.Error as e:
        with open(f'sist_oppdatert_{tabell}.txt', 'w') as f_out:
            f_out.write(dato)
            logging.debug(f"{tabell} er sist oppdatert (lokal): {dato}")
    return None

In [6]:
def akv_hent_CD2_access_token():
    be_om_access_token = requests.request(
        "POST",
        f"{CD2_base_url}/ids/auth/login",
        data={'grant_type': 'client_credentials'},
        auth=(CD2_client_id, CD2_client_secret)
        )
    if be_om_access_token.status_code == 200:
        CD2_access_token = be_om_access_token.json()['access_token']
        return CD2_access_token
    else:
        feilmelding = f"Klarte ikkje å skaffe access_token, feil {be_om_access_token.status_code}"
        logging.error(feilmelding)
        return feilmelding


def akv_hent_CD2_filar(innfil, token, svar):
    requesturl = f"{CD2_base_url}/dap/object/url"
    payload = f"{svar['objects']}"
    payload = payload.replace('\'', '\"')
    headers = {'x-instauth': token, 'Content-Type': 'text/plain'}
    r4 = requests.request("POST", requesturl, headers=headers, data=payload)
    if r4.status_code == 200:
        respons4 = r4.json()
        url = respons4['urls'][innfil]['url']
        data = requests.request("GET", url)
        buffer = io.BytesIO(data.content)
        with gzip.GzipFile(fileobj=buffer, mode='rb') as utpakka_fil:
            utpakka_data = utpakka_fil.read().decode()
    return utpakka_data

In [7]:
def akv_les_CD2_tabell(tabell):
    CD2_access_token = akv_hent_CD2_access_token()
    headers = {'x-instauth': CD2_access_token, 'Content-Type': 'text/plain'}
    sist_oppdatert = akv_finn_sist_oppdatert(tabell)
    payload = '{"format": "csv", "since": \"%s\"}' % (sist_oppdatert)
    requesturl = f"{CD2_base_url}/dap/query/canvas/table/{tabell}/data"
    print(f"Sender søk til {requesturl}")
    try:
        r = requests.request("POST", requesturl, headers=headers, data=payload)
        r.raise_for_status()
        respons = r.json()
        id = respons['id']
        vent = True
        while vent:
            requesturl2 = f"{CD2_base_url}/dap//job/{id}"
            r2 = requests.request("GET", requesturl2, headers=headers)
            time.sleep(5)
            respons2 = r2.json()
            print(respons2)
            if respons2['status'] == "complete":
                vent = False
                filar = respons2['objects']
        dr_liste = []
        print(filar)
        for fil in filar:
            data = io.StringIO(akv_hent_CD2_filar(fil['id'], CD2_access_token, respons2))
            df = pd.read_csv(data, sep=",")
            dr_liste.append(df)
        alledata = pd.concat(df for df in dr_liste if not df.empty)
        return alledata, sist_oppdatert, respons2['until']
    except requests.exceptions.RequestException as exc:
        raise exc

In [8]:
tabell = "Canvas_Enrollments"
if os.path.exists(f'loggfil-{tabell}.log'):
    os.remove(f'loggfil-{tabell}.log')

# Opprett logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)  # Sett ønsket loggnivå

# Opprett formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Opprett filhandler for å logge til fil
file_handler = logging.FileHandler(f'loggfil-{tabell}.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

# Opprett konsollhandler for å logge til konsollen
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

# Legg til handlerne i loggeren
logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [9]:
start_Canvas_Enrollments = time.perf_counter()
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    try:
        query = """
        SELECT ALL * FROM [stg].[Canvas_Terms]
        """
        cursor.execute(query)
        row = cursor.fetchall()
        terminar = []
        for t in row:
            try:
                term_id = t[0]
                name = t[1]
                start_at = t[2]
                end_at = t[3]
                created_at = t[4]
                terminar.append([term_id, name, start_at, end_at, created_at])
            except IndexError:
                logger.error("Ingen data i denne raden: {t}")
    except pyodbc.Error as e:
        logger.error(f"Feil: {e}")

In [10]:

aktuelle_terminar = []
try:
    for t in terminar:
        desimal = int(år) + 0.5*(termin == 'HØST')
        if '-' in t[1]:
            start_termin = t[1].split('-')[0]
            slutt_termin = t[1].split('-')[1]
            start_år = start_termin.split(' ')[0]
            slutt_år = slutt_termin.split(' ')[0]
            start_semester = start_termin.split(' ')[1]
            slutt_semester = slutt_termin.split(' ')[1]
            start_desimal = int(start_år) + 0.5*(start_semester == 'HØST')
            slutt_desimal = int(slutt_år) + 0.5*(slutt_semester == 'HØST')
            if start_desimal <= desimal <= slutt_desimal:
                aktuelle_terminar.append(t[0])
            else:
                pass
        elif t[1] == f"{str(år)} {termin}":
            aktuelle_terminar.append(t[0])
        else:
            pass
except:
    logger.error(f"Feil: {traceback.format_exc()}")

In [11]:
with pyodbc.connect(conn_str) as conn:
    cursor = conn.cursor()
    try:
        query = """
        SELECT ALL * FROM [stg].[Canvas_Courses]
        """
        cursor.execute(query)
        row = cursor.fetchall()
        aktuelle_emne = []
        for emne in row:
            try:
                if emne[4] in aktuelle_terminar:
                    aktuelle_emne.append(emne[0])
            except IndexError:
                pass
    except pyodbc.Error as e:
        logger.error(f"Feil: {e}")

In [12]:
tabell = "enrollments"
resultat = akv_les_CD2_tabell(tabell)

Sender søk til https://api-gateway.instructure.com/dap/query/canvas/table/enrollments/data
{'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342', 'status': 'running', 'expires_at': '2024-11-28T19:50:58Z'}
{'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342', 'status': 'running', 'expires_at': '2024-11-28T19:50:58Z'}
{'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342', 'status': 'complete', 'objects': [{'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342/part-00000-fdb631db-375e-4f35-9fc7-a10ebf0aef4c-c000.csv.gz'}, {'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342/part-00005-fdb631db-375e-4f35-9fc7-a10ebf0aef4c-c000.csv.gz'}], 'expires_at': '2024-11-28T19:50:58Z', 'schema_version': 1, 'since': '2024-10-24T07:06:15Z', 'until': '2024-11-27T19:31:42Z'}
[{'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342/part-00000-fdb631db-375e-4f35-9fc7-a10ebf0aef4c-c000.csv.gz'}, {'id': '9f29fd57-d90a-4f79-80e2-32bf6384d342/part-00005-fdb631db-375e-4f35-9fc7-a10ebf0aef4c-c000.csv.gz'}]


/tmp/ipykernel_32791/3018563503.py:27: DtypeWarning: Columns (14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data, sep=",")


In [13]:
enrollments = resultat[0][['key.id', 'value.user_id', 'value.course_id', 'value.type', 'value.created_at', 'value.updated_at', 'value.workflow_state', 'value.total_activity_time', 'value.last_activity_at']]
enrollments['sis_user_id'] = ''

/tmp/ipykernel_32791/2303297184.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrollments['sis_user_id'] = ''


In [14]:
enrollments

,key.id,value.user_id,value.course_id,value.type,value.created_at,value.updated_at,value.workflow_state,value.total_activity_time,value.last_activity_at,sis_user_id
0,1256959,67355,25541,StudentEnrollment,2023-12-17T02:29:49.872Z,2024-09-24T09:56:44.315Z,active,73262.0,2024-06-10T14:04:26.182Z,
1,1282347,47509,25834,StudentEnrollment,2024-01-07T07:42:53.679Z,2024-09-24T10:00:03.357Z,active,67998.0,2024-05-31T07:54:14.812Z,
2,1262583,94554,25541,StudentEnrollment,2023-12-19T16:12:55.612Z,2024-09-24T09:40:55.958Z,active,277491.0,2024-11-14T12:34:38.652Z,
3,1249218,79426,24602,StudentEnrollment,2023-12-15T06:12:21.014Z,2024-09-24T09:45:33.084Z,active,193122.0,2024-10-28T11:18:08.898Z,
4,1256585,83726,25881,StudentEnrollment,2023-12-16T16:14:38.638Z,2024-09-24T09:43:09.498Z,active,502315.0,2024-11-10T08:58:44.358Z,
...,...,...,...,...,...,...,...,...,...,...
219982,1456010,4945,29436,TeacherEnrollment,2024-11-27T15:01:25.373Z,2024-11-27T16:46:11.996Z,active,NaN,2024-11-27T16:47:40.923Z,
219983,1456021,4939,30808,TeacherEnrollment,2024-11-27T16:12:07.850Z,2024-11-27T16:12:07.850Z,active,NaN,NaN,
219984,1456020,5081,30808,TeacherEnrollment,2024-11-27T16:12:07.768Z,2024-11-27T16:12:07.768Z,active,NaN,NaN,
219985,1456023,109430,30807,TeacherEnrollment,2024-11-27T18:29:53.128Z,2024-11-27T18:29:53.128Z,invited,NaN,NaN,
